In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd 
import numpy as np
import time
from sentence_transformers import SentenceTransformer,util
import tensorflow
import pickle

In [3]:
from fobokiller.cosim import load_embedding , compute_sim_df ,model

In [4]:
embedding = load_embedding()

2021-11-26 17:15:57.097352: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
df = compute_sim_df('I want to eat a pizza in a cosy place',np.array(embedding), )
df['ratio']=df['review_filtered']/df['review_all']
df
######
#sim_r = distance moyenne par restau des reviews qui sortent
#sim_s = distance review
#review_filtered = nbr de review qui sortent
#review_all = total review par restau
#ratio = review_filtered/review_all
######

,alias,date,rate,review_clean,review_sentences,sim_s,rate_filtered,review_filtered,sim_r,rate_all,review_all,ratio
0,pizza-julia-paris,2015-08-06,5,perfect place for some good pizza -- fresh ing...,perfect place for some good pizza -- fresh ing...,0.663496,5.0,1,0.663496,4.538860,60,0.016667
1,firmine-paris,2018-08-16,5,great pizzas! we stopped here after we went on...,must place to eat if you love pizza,0.658819,4.5,2,0.638282,3.903250,372,0.005376
2,falstaff-paris-2,2011-04-24,1,i just arrived in paris and was starved! the f...,"if you want pizza, go across the street to the...",0.655241,1.0,1,0.655241,2.010753,10,0.100000
3,swann-et-vincent-paris-2,2015-05-07,4,the place is really nice and the food is great...,a real comfortable italian restaurant,0.652533,4.0,1,0.652533,4.085714,4,0.250000
4,carmine-café-paris-3,2017-10-20,4,pizzas somewhere . out of . i would change the...,pizzas somewhere,0.637849,4.0,1,0.637849,4.003279,80,0.012500
5,east-mamma-paris-5,2018-05-04,4,good pizza and pasta delicious all waiters fri...,yamy yamy pizza good place for pizza,0.627936,4.5,2,0.617330,4.386667,76,0.026316
6,ozio-paris,2019-07-15,5,had the campione pizza. ok.... i know... why w...,: this place has delicious pizza,0.623461,3.0,2,0.614814,4.137634,66,0.030303
7,il-gigolo-paris,2020-01-13,5,the pizza here is so good! i love how the crus...,the pizza here is so good,0.623159,5.0,1,0.623159,4.545455,10,0.100000
8,pedzouille-paris,2019-05-27,5,this is awesome. we love the food and the serv...,if you want great food in a cozy restaurant,0.622159,5.0,1,0.622159,5.000000,13,0.076923
9,il-sorrentino-paris,2015-07-08,5,i wanted to find a good non-touristy italian r...,a cozy place with great italian food and the ...,0.618599,5.0,1,0.618599,4.248705,49,0.020408


In [40]:
df['ratio_x_sim'] = df['ratio']*df['sim_r']

In [41]:
ratio_x_sim = df.groupby('alias').sum().sort_values(by = 'ratio_x_sim', 
                                      ascending=False)[:20].index

ratio = df.groupby('alias').sum().sort_values(by = 'ratio', 
                                      ascending=False)[:20].index
sim = df.groupby('alias').sum().sort_values(by = 'sim_r', 
                                      ascending=False)[:20].index
                                                                              

In [42]:
pd.DataFrame([sim,ratio, ratio_x_sim], 
             index = ["sim","ratio", "ratio_x_sim"]).T

,sim,ratio,ratio_x_sim
0,pizza-julia-paris,pizza-julia-paris,pizza-julia-paris
1,firmine-paris,pizzeria-popolare-paris-2,pizzeria-popolare-paris-2
2,gambino-paris-2,la-briciola-paris,la-briciola-paris
3,pizza-pino-paris-8,hank-paris-5,hank-paris-5
4,pizzeria-babalou-paris,firmine-paris,firmine-paris
5,pizzeria-popolare-paris-2,pizzeria-babalou-paris,grazie-paris
6,hank-paris-5,grazie-paris,pizzeria-babalou-paris
7,ober-mamma-paris,il-brigante-paris,il-brigante-paris
8,la-briciola-paris,mokus-l-ecureuil-paris,mokus-l-ecureuil-paris
9,mokus-l-ecureuil-paris,pizza-di-loretta-paris,pizza-di-loretta-paris


In [43]:
df['restau_output'] =  df['ratio']*df['sim_r']*df['rate_filtered']/5

In [44]:
pd.DataFrame([df.groupby('alias').sum().sort_values(by = 'restau_output', 
                                      ascending=False)[:20].index, ratio_x_sim ], index=['metrics_rank','output']).T

,metrics_rank,output
0,pizza-julia-paris,pizza-julia-paris
1,la-briciola-paris,pizzeria-popolare-paris-2
2,pizzeria-popolare-paris-2,la-briciola-paris
3,hank-paris-5,hank-paris-5
4,pizzeria-babalou-paris,firmine-paris
5,grazie-paris,grazie-paris
6,firmine-paris,pizzeria-babalou-paris
7,il-brigante-paris,il-brigante-paris
8,pizza-di-loretta-paris,mokus-l-ecureuil-paris
9,mokus-l-ecureuil-paris,pizza-di-loretta-paris


In [21]:
liste = [
    "I want a pizza gluten free",
    "pizza gluten free",
    "best pizza gluten free",
    "I want the best pizza gluten free",
    "I want a burger",
    "I want a burger in a quiet place",
    "I want the best burger",
    "I want the best burger in a quiet place",
    "burger quiet",
    "best burger",
    "I feel like eating ramens today!",
    "Best ramens",
    "best ramens",
    "ramens",
    "I’d like to brunch at a nice place with pancakes and great coffee",
    "I would like to brunch at a nice place with pancakes and great coffee",
    "pancakes greate coffee",
    "I want to eat in a traditional Japanese restaurant with floor seats.",
    "eat in a traditional Japanese restaurant with floor seats.",
    "I would like to eat with loud music  !"
    "eat loud music",
    "I would like some typical sushis not too expensive",
    "some typical sushis not too expensive",
    "I'd love some salmon lasagna",
    "some salmon lasagna",
    "I'd like a good steak!",
    "I would like a good steak",
    "a good steak",
    "I want to eat a fresh salad with shrimps and nuts.",
    "fresh salad shrimps nuts"
    
    
]

In [25]:

def generate_result(sentence) :
    print("en cours : -- " + sentence)
    df = compute_sim_df(sentence,np.array(embedding), n_prox = 1000 )
    df['ratio']=df['review_filtered']/df['review_all']
    df.to_csv(sentence.replace(" ","_") + ".csv")

In [26]:
%%time
[generate_result(i) for i in liste]

en cours : -- I want a pizza gluten free
en cours : -- pizza gluten free
en cours : -- best pizza gluten free
en cours : -- I want the best pizza gluten free
en cours : -- I want a burger
en cours : -- I want a burger in a quiet place
en cours : -- I want the best burger
en cours : -- I want the best burger in a quiet place
en cours : -- burger quiet
en cours : -- best burger
en cours : -- I feel like eating ramens today!
en cours : -- Best ramens
en cours : -- best ramens
en cours : -- ramens
en cours : -- I’d like to brunch at a nice place with pancakes and great coffee
en cours : -- I would like to brunch at a nice place with pancakes and great coffee
en cours : -- pancakes greate coffee
en cours : -- I want to eat in a traditional Japanese restaurant with floor seats.
en cours : -- eat in a traditional Japanese restaurant with floor seats.
en cours : -- I would like to eat with loud music  !eat loud music
en cours : -- I would like some typical sushis not too expensive
en cours : -

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]